# Hello World Redis


In [1]:
!pip install redis

In [7]:
import redis
r = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)
r.set('name', 'Mohammad')
value = r.get('name')
print(value)


Mohammad


# Store User Session Information

In [8]:
def store_user_session(user_id, last_login):
    session_key = f"user_session:{user_id}"
    r.hmset(session_key, {"user_id": user_id, "last_login": last_login})
    print(f"Session user {user_id} successfully.")

def retrieve_user_session(user_id):
    session_key = f"user_session:{user_id}"
    session_data = r.hgetall(session_key)
    if session_data:
        print(f"Session data retrieved for user {user_id}: {session_data}")
    else:
        print(f"No session data found for user {user_id}.")
    return session_data

if __name__ == "__main__":
    user_id = "123"
    last_login = "2024-09-15 10:00:00"
    store_user_session(user_id, last_login)
    retrieve_user_session(user_id)

Session data for user 123 stored successfully.
Session data retrieved for user 123: {'user_id': '123', 'last_login': '2024-09-15 10:00:00'}


C:\Users\nematizadeh\AppData\Local\Temp\ipykernel_8880\3965061049.py:10: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset(session_key, {"user_id": user_id, "last_login": last_login})


# Expire Keys

In [10]:
import redis
import time

def store_otp(user_id, otp, expiration=60):
    otp_key = f"otp:{user_id}"
    r.setex(otp_key, expiration, otp)

def check_otp(user_id):
    otp_key = f"otp:{user_id}"
    otp_value = r.get(otp_key)
    if otp_value:
        print(f"OTP for user {user_id} is still available: {otp_value}")
    else:
        print(f"OTP for user {user_id} has expired or does not exist.")
    return otp_value

if __name__ == "__main__":
    user_id = "user_123"
    otp = "987654"
    store_otp(user_id, otp, expiration=60)
    check_otp(user_id)
    print("Waiting for 60 seconds...")
    time.sleep(60)
    check_otp(user_id)


OTP for user user_123 is still available: 987654
Waiting for 60 seconds...
OTP for user user_123 has expired or does not exist.


# Leaderboard Implementation

In [11]:

def add_users_scores(users_scores):
    leaderboard_key = "leaderboard"
    for user, score in users_scores.items():
        r.zadd(leaderboard_key, {user: score})
    print("Users and scores added to leaderboard.")

def get_top_n_users(n):
    leaderboard_key = "leaderboard"
    
    top_users = r.zrevrange(leaderboard_key, 0, n-1, withscores=True)
    
    return top_users

if __name__ == "__main__":
    users_scores = {
        'user1': 50,
        'user2': 80,
        'user3': 40,
        'user4': 90
    }
    add_users_scores(users_scores)
    top_3_users = get_top_n_users(3)
    print("Top 3 users:")
    for user, score in top_3_users:
        print(f"{user}: {score}")


Users and scores added to leaderboard.
Top 3 users:
user4: 90.0
user2: 80.0
user1: 50.0


# Batch User Data Updates with Redis Pipelines

In [12]:
users_data = [
    {"user_id": "user_1", "age": 25, "score": 85.5},
    {"user_id": "user_2", "age": 30, "score": 90.0},
    {"user_id": "user_3", "age": 22, "score": 76.4},
]
pipe = r.pipeline()
for user in users_data:
    user_key = f"user:{user['user_id']}"
    pipe.hmset(user_key, {"age": user["age"], "score": user["score"]})

pipe.execute()

for user in users_data:
    user_key = f"user:{user['user_id']}"
    user_data = r.hgetall(user_key)
    print(f"{user_key} -> {user_data}")


user:user_1 -> {'age': '25', 'score': '85.5'}
user:user_2 -> {'age': '30', 'score': '90.0'}
user:user_3 -> {'age': '22', 'score': '76.4'}


C:\Users\nematizadeh\AppData\Local\Temp\ipykernel_8880\2443223210.py:14: DeprecationWarning: Pipeline.hmset() is deprecated. Use Pipeline.hset() instead.
  pipe.hmset(user_key, {"age": user["age"], "score": user["score"]})
